# How to Forecast a Time Series with Python

Wouldn't it be nice to know the future? This is the notebook that relates to the blog post on medium. Please check the blog for visualizations and explanations, this notebook is really just for the code :)


## Processing the Data

Let's explore the Industrial production of electric and gas utilities in the United States, from the years 1985-2018, with our frequency being Monthly production output.

You can access this data here: https://fred.stlouisfed.org/series/IPG2211A2N

This data measures the real output of all relevant establishments located in the United States, regardless of their ownership, but not those located in U.S. territories.

In [1]:
%matplotlib inline
import pandas as pd
data = pd.read_csv("Electric_Production.csv",index_col=0)
data.head()

,IPG2211A2N
DATE,
1985-01-01,72.5052
1985-02-01,70.6720
1985-03-01,62.4502
1985-04-01,57.4714
1985-05-01,55.3151


Right now our index is actually just a list of strings that look like a date, we'll want to adjust these to be timestamps, that way our forecasting analysis will be able to interpret these values:

In [2]:
data.index

Index(['1985-01-01', '1985-02-01', '1985-03-01', '1985-04-01', '1985-05-01',
       '1985-06-01', '1985-07-01', '1985-08-01', '1985-09-01', '1985-10-01',
       ...
       '2017-04-01', '2017-05-01', '2017-06-01', '2017-07-01', '2017-08-01',
       '2017-09-01', '2017-10-01', '2017-11-01', '2017-12-01', '2018-01-01'],
      dtype='object', name='DATE', length=397)

In [3]:
data.index = pd.to_datetime(data.index)

In [4]:
data.head()

,IPG2211A2N
DATE,
1985-01-01,72.5052
1985-02-01,70.6720
1985-03-01,62.4502
1985-04-01,57.4714
1985-05-01,55.3151


In [5]:
data.index

DatetimeIndex(['1985-01-01', '1985-02-01', '1985-03-01', '1985-04-01',
               '1985-05-01', '1985-06-01', '1985-07-01', '1985-08-01',
               '1985-09-01', '1985-10-01',
               ...
               '2017-04-01', '2017-05-01', '2017-06-01', '2017-07-01',
               '2017-08-01', '2017-09-01', '2017-10-01', '2017-11-01',
               '2017-12-01', '2018-01-01'],
              dtype='datetime64[ns]', name='DATE', length=397, freq=None)

Let's first make sure that the data doesn't have any missing data points:

In [6]:
data[pd.isnull(data['IPG2211A2N'])]

,IPG2211A2N
DATE,


Let's also rename this column since its hard to remember what "IPG2211A2N" code stands for:

In [7]:
data.columns = ['Energy Production']

In [8]:
data.head()

,Energy Production
DATE,
1985-01-01,72.5052
1985-02-01,70.6720
1985-03-01,62.4502
1985-04-01,57.4714
1985-05-01,55.3151


In [9]:
import plotly
# plotly.tools.set_credentials_file()

In [27]:
from plotly.plotly import plot_mpl
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(data, model='multiplicative')
fig = result.plot()
plot_mpl(fig)

'https://plot.ly/~Pierian-Data/12'

In [28]:
import plotly.plotly as ply
import cufflinks as cf
# Check the docs on setting up offline plotting

In [ ]:
data.iplot(title="Energy Production Jan 1985--Jan 2018", theme='pearl')

## Train Test Split

In [9]:
data.head()

,Energy Production
DATE,
1985-01-01,72.5052
1985-02-01,70.6720
1985-03-01,62.4502
1985-04-01,57.4714
1985-05-01,55.3151


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 397 entries, 1985-01-01 to 2018-01-01
Data columns (total 1 columns):
Energy Production    397 non-null float64
dtypes: float64(1)
memory usage: 6.2 KB


We'll train on 20 years of data, from the years 1985-2015 and test our forcast on the years after that and compare it to the real data.

In [11]:
train = data.loc['1985-01-01':'2014-12-01']

In [12]:
train.tail()

,Energy Production
DATE,
2014-08-01,108.1940
2014-09-01,100.4172
2014-10-01,92.3837
2014-11-01,99.7033
2014-12-01,109.3477


In [13]:
test = data.loc['2015-01-01':]

In [14]:
test.head()

,Energy Production
DATE,
2015-01-01,120.2696
2015-02-01,116.3788
2015-03-01,104.4706
2015-04-01,89.7461
2015-05-01,91.0930


In [15]:
test.tail()

,Energy Production
DATE,
2017-09-01,98.6154
2017-10-01,93.6137
2017-11-01,97.3359
2017-12-01,114.7212
2018-01-01,129.4048


In [16]:
len(test)

37

In [17]:
from pyramid.arima import auto_arima

**he AIC measures how well a model fits the data while taking into account the overall complexity of the model. A model that fits the data very well while using lots of features will be assigned a larger AIC score than a model that uses fewer features to achieve the same goodness-of-fit. Therefore, we are interested in finding the model that yields the lowest AIC value.

In [18]:
stepwise_model = auto_arima(train, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=12,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True) 

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=1562.673, BIC=1581.920, Fit time=0.810 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=1760.323, BIC=1768.021, Fit time=0.023 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=1700.037, BIC=1715.434, Fit time=0.234 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=1616.434, BIC=1631.831, Fit time=0.249 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=1563.343, BIC=1586.439, Fit time=1.089 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=1678.972, BIC=1694.369, Fit time=0.468 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 2, 12); AIC=1562.675, BIC=1585.771, Fit time=3.676 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=1561.966, BIC=1588.911, Fit time=3.803 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=1616.436, BIC=1639.532, Fit time=1.695 seconds
Fit ARIMA: order=(2, 1, 1) s

In [19]:
stepwise_model.aic()

1557.4705689357481

In [20]:
future_forecast = stepwise_model.predict(n_periods=len(test))

In [21]:
future_forecast

array([118.03491147, 108.7540358 , 100.4580758 ,  89.87684977,
        92.28568729, 102.61446451, 111.59283657, 110.72327607,
        99.7325228 ,  91.87267577,  96.39353831, 109.49430632,
       117.42272981, 108.22115232, 100.57831178,  90.27458144,
        92.23678195, 102.51126709, 111.95153831, 110.762962  ,
        99.62040835,  91.85119294,  96.07617065, 110.35951153,
       118.70652065, 109.08683149, 101.45466743,  90.37675255,
        92.23242288, 102.38626699, 111.35758369, 110.26903977,
        99.748839  ,  91.93560549,  96.7055109 , 110.27813967,
       118.75490027])

In [22]:
future_forecast = pd.DataFrame(future_forecast,index = test.index,columns=['Prediction'])

In [23]:
future_forecast.head()

,Prediction
DATE,
2015-01-01,118.034911
2015-02-01,108.754036
2015-03-01,100.458076
2015-04-01,89.876850
2015-05-01,92.285687


In [24]:
test.head()

,Energy Production
DATE,
2015-01-01,120.2696
2015-02-01,116.3788
2015-03-01,104.4706
2015-04-01,89.7461
2015-05-01,91.0930


In [26]:
pd.concat([test,future_forecast],axis=1).iplot()

AttributeError: 'DataFrame' object has no attribute 'iplot'

In [48]:
pd.concat([data,future_forecast],axis=1).iplot()